<a href="https://colab.research.google.com/github/Jyunci/AIoT_cute/blob/main/TCN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 手寫數字辨識
使用 TCN (Temporal Convolution Network) 模型進行手寫數字的辨識

Ref: https://github.com/ashishpatel26/tcn-keras-Examples

In [1]:
!pip install keras-tcn

     |████████████████████████████████| 378kB 9.6MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [2]:
import numpy as np
from keras.datasets import mnist
from keras.utils import to_categorical
from tcn import compiled_tcn

Using TensorFlow backend.


In [3]:
def data_generator():
    # input image dimensions
    img_rows, img_cols = 28, 28
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(-1, img_rows * img_cols, 1)
    x_test = x_test.reshape(-1, img_rows * img_cols, 1)

    num_classes = 10
    y_train = to_categorical(y_train, num_classes)
    y_test = to_categorical(y_test, num_classes)

    y_train = np.expand_dims(y_train, axis=2)
    y_test = np.expand_dims(y_test, axis=2)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255

    return (x_train, y_train), (x_test, y_test)

In [4]:
(x_train, y_train), (x_test, y_test) = data_generator()

11493376/11490434 [==============================] - 1s 0us/step


## 設計 TCN (Temporal Convolutional Network) 模型

In [5]:
 model = compiled_tcn(return_sequences=False,
                         num_feat=1,
                         num_classes=10,
                         nb_filters=20,
                         kernel_size=6,
                         dilations=[2 ** i for i in range(9)],
                         nb_stacks=1,
                         max_len=x_train[0:1].shape[1],
                         use_skip_connections=True)

x.shape= (None, 20)
model.x = (None, 784, 1)
model.y = (None, 10)


In [6]:
print(f'x_train.shape = {x_train.shape}')
print(f'y_train.shape = {y_train.shape}')
print(f'x_test.shape = {x_test.shape}')
print(f'y_test.shape = {y_test.shape}')

x_train.shape = (60000, 784, 1)
y_train.shape = (60000, 10, 1)
x_test.shape = (10000, 784, 1)
y_test.shape = (10000, 10, 1)


In [7]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784, 1)]          0         
_________________________________________________________________
tcn (TCN)                    (None, 20)                41320     
_________________________________________________________________
dense (Dense)                (None, 10)                210       
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 41,530
Trainable params: 41,530
Non-trainable params: 0
_________________________________________________________________


## 模型訓練

In [9]:
model.fit(x_train, y_train.squeeze().argmax(axis=1), epochs=5,validation_data=(x_test, y_test.squeeze().argmax(axis=1)))

Epoch 1/5
1875/1875 [==============================] - 34s 18ms/step - loss: 0.2141 - accuracy: 0.9350 - val_loss: 0.1254 - val_accuracy: 0.9611
Epoch 2/5
1875/1875 [==============================] - 35s 18ms/step - loss: 0.1441 - accuracy: 0.9575 - val_loss: 0.0863 - val_accuracy: 0.9729
Epoch 3/5
1875/1875 [==============================] - 34s 18ms/step - loss: 0.1161 - accuracy: 0.9646 - val_loss: 0.0868 - val_accuracy: 0.9748
Epoch 4/5
1875/1875 [==============================] - 34s 18ms/step - loss: 0.1059 - accuracy: 0.9685 - val_loss: 0.0785 - val_accuracy: 0.9748
Epoch 5/5
1875/1875 [==============================] - 34s 18ms/step - loss: 0.0997 - accuracy: 0.9699 - val_loss: 0.0885 - val_accuracy: 0.9753


## 模型評估

In [13]:
loss, accuracy = model.evaluate(x_test, y_test.squeeze().argmax(axis=1))
print('Test:')
print('Loss: %s\nAccuracy: %s' % (loss, accuracy))

313/313 [==============================] - 2s 6ms/step - loss: 0.0885 - accuracy: 0.9753
Test:
Loss: 0.08851510286331177
Accuracy: 0.9753000140190125
